In [1]:
import spacy

from spacy import displacy
from spacy.tokens import Span, DocBin

from spacy.displacy.render import DEFAULT_LABEL_COLORS

print(f'spaCy version: {spacy.__version__}')

# Load the trained model
nlp = spacy.load('./output/model-best')

spaCy version: 3.8.3


In [4]:
doc_bin = DocBin().from_disk("./training/test.spacy")
docs = list(doc_bin.get_docs(nlp.vocab))

[MIGROS
 
 Genossenschaft Migros Luzern
 SportXX Surseepark
 Tel. 041 926 39 69
 
 Artikelbezeichnung Menge Preis Gespart Total
 
 PB Electrolyte Tabs 3 5.90 17.70
 Sponser Electrolytes T 1 4.90 4.90
 SPONSER Electrolyte ] 4.90
 SPONSER Electrolyte
 
 Total CHF
 
 Mastercard
 Total in EUR
 
 Buchung HASTERCARD Cuntactlass
 XKKKKRKKRRAK 1746
 
 27.09.2022 12:22
 #31424570x005 145037073899 70000
 00000028
 
 Total-EFT CHF:
 
 Cunulus-Nunner
 Punktestand per 23.09.2022
 Erhaltene Punkte
 
 Ar
 ,
 (fv) Die Gastronormi CRP UP pe
 
 Hochschule Luzern
 Informatik 8 Wirtschaft
 cfo ZFV-Unternehmungen
 Fluelastrasse 51
 8047 Zurich
 
 CHF
 Pizza Neapolitana 10.50
 
 Ramsejer Apfelschor) 3,00
 
 Total CHF [Syf 0
 
 2 MASTERCARD Hi, 0
 
 MwSt Prozent Netto Brutto MwSt
 
 (1) 8.10 12,49 13.50 1.01
 Profit Center: InHouse
 29.11.2024 11:36:49 #:2459 0p:568071 ¢C:56
 B002 $:568
 Es bediente Sie : Sel fCheck
 
 Kopie # ]
 
 CHE-105.827. 102 MWST
 Vielen Dank und auf Wiedersehen
 ,
 DUIV Kriens 0
 Tel

In [12]:
# Example input text
text = docs[1]

# Process the text with the trained model
doc = nlp(text)

In [13]:
print(text)

(fv) Die Gastronormi CRP UP pe

Hochschule Luzern
Informatik 8 Wirtschaft
cfo ZFV-Unternehmungen
Fluelastrasse 51
8047 Zurich

CHF
Pizza Neapolitana 10.50

Ramsejer Apfelschor) 3,00

Total CHF [Syf 0

2 MASTERCARD Hi, 0

MwSt Prozent Netto Brutto MwSt

(1) 8.10 12,49 13.50 1.01
Profit Center: InHouse
29.11.2024 11:36:49 #:2459 0p:568071 ¢C:56
B002 $:568
Es bediente Sie : Sel fCheck

Kopie # ]

CHE-105.827. 102 MWST
Vielen Dank und auf Wiedersehen




In [14]:
# Access the predicted spans
print("Predicted Spans:")
for span in doc.spans["sc"]:  # Use the spans_key defined in your config
    print(f"{span.label_}: {span.text}")

Predicted Spans:
RECEIPT_ITEM_PRICE: 10.50
RECEIPT_ITEM_PRICE: 3,00
RECEIPT_ITEM_NAME: )
RECEIPT_ITEM_PRICE: 1.01
RECEIPT_ITEM_NAME: Pizza Neapolitana
RECEIPT_ITEM: Pizza Neapolitana 10.50


In [15]:
labels = set()
for s in doc.spans["sc"]:
    labels.add(s.label_)

colour_dict = {}
colours = list(DEFAULT_LABEL_COLORS.values())
for i, t in enumerate(labels):
    colour_dict[t] = colours[i]

In [16]:
options = {"colors": colour_dict}

displacy.render(doc, style='span', jupyter=True, options=options)